<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/LLM_Court.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import time
import re
import openai
import json
from sklearn.model_selection import train_test_split
import ast

In [2]:
def fix_hebrew_labels(ax):
    """ Reverse Hebrew labels in Matplotlib plots. """
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        text = label.get_text()
        if any("\u0590" <= char <= "\u05FF" for char in text):  # Detect Hebrew characters
            label.set_text(text[::-1])
    ax.figure.canvas.draw()


In [3]:

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"
classifiers_path = "/content/drive/MyDrive/בית המשפט - מסווגים/"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


All Excel files have been concatenated successfully!


In [ ]:
full_ra_rap = pd.read_excel("/content/drive/My Drive/full_ra_rap.xlsx")

In [ ]:
check_gender_bias = pd.concat([dataframes[0], dataframes[1], dataframes[2]])
check_gender_bias = pd.merge(full_ra_rap, check_gender_bias, on='מספר הליך', how='left')
check_gender_bias.drop_duplicates(subset = 'מספר הליך',inplace = True)
check_gender_bias.dropna(subset = ["בקשה לרשות ערעור התקבלה?"],inplace=True)

In [ ]:
for col in check_gender_bias.columns:
    if(col.find("מגדר")!=-1 or col.find("ערעור")!=-1 or col.find("מספר ")!=-1):
      print(col)

In [ ]:
check_gender_bias = check_gender_bias[["מספר הליך","מספר השופטים","בקשה לרשות ערעור התקבלה?","מגדר"]]

In [ ]:
check_gender_bias[["בקשה לרשות ערעור התקבלה?","מגדר"]]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval


# המרת המחרוזת ברשימת מגדרים למבנה רשימה
check_gender_bias["מגדר"] = check_gender_bias["מגדר"].apply(literal_eval)  # ממיר טקסט לרשימה

# פונקציה להשטחת הנתונים – כל שופט יקבל שורה משלו
def expand_genders(df):
    expanded_data = []
    for _, row in df.iterrows():
        for gender in row["מגדר"]:
            expanded_data.append({"Gender": gender, "Appeal Outcome": row["בקשה לרשות ערעור התקבלה?"]})
    return pd.DataFrame(expanded_data)

# הרחבת הטבלה
df_expanded = expand_genders(check_gender_bias)

# ספירת תוצאות הערעור לפי מגדר
outcome_counts = df_expanded.groupby(["Gender", "Appeal Outcome"]).size().unstack().fillna(0)

# יצירת גרף התפלגות תוצאות לפי מגדר
plt.figure(figsize=(8, 5))
outcome_counts.plot(kind="bar", stacked=True)
plt.ylabel("מספר ערעורים")
plt.title("התפלגות תוצאות הערעור לפי מגדר השופט/ת")
plt.xticks(rotation=0)
plt.legend(title="תוצאת הערעור")
plt.show()

# חישוב יחס קבלת ערעור לכל מגדר
outcome_counts["Total"] = outcome_counts.sum(axis=1)
outcome_counts["Acceptance Rate"] = outcome_counts.get("התקבל", 0) / outcome_counts["Total"]

# יצירת גרף של אחוזי קבלת ערעור לפי מגדר
plt.figure(figsize=(8, 5))
sns.barplot(x=outcome_counts.index, y=outcome_counts["Acceptance Rate"])
plt.ylabel("שיעור קבלת ערעור")
plt.title("שיעור קבלת ערעור לפי מגדר השופט/ת")
plt.ylim(0, 1)
plt.show()

# הצגת הטבלה עם התוצאות
print(outcome_counts)


# Bias - number of Judges

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval

# טעינת הנתונים
df = check_gender_bias.copy()

# חישוב מספר השופטים בכל תיק
df["מספר שופטים"] = df["מגדר"].apply(len)

# חלוקת המקרים לשתי קבוצות: שופט יחיד או יותר מאחד
df["יותר משופט אחד"] = df["מספר שופטים"] > 1

# ספירת המקרים לפי מספר השופטים והאם הערעור התקבל
outcome_counts = df.groupby("יותר משופט אחד")["בקשה לרשות ערעור התקבלה?"].value_counts(normalize=True).unstack()

# יצירת גרף
plt.figure(figsize=(8, 5))
outcome_counts.plot(kind="bar", stacked=True)
plt.ylabel("שיעור הערעורים (%)")
plt.title("השפעת מספר השופטים על קבלת הערעור")
plt.xticks([0, 1], ["שופט אחד", "יותר משופט אחד"], rotation=0)
plt.legend(title="תוצאת הערעור")
plt.show()

# חישוב אחוז קבלת ערעור לפי מספר השופטים
acceptance_rate = outcome_counts.get("התקבל", 0)

# הדפסת סטטיסטיקה
print("שיעור קבלת הערעור לפי מספר שופטים:")
print(acceptance_rate)


In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

df = check_gender_bias.copy()

# לוודא שהעמודה 'מגדר' היא רשימה ולא מחרוזת
df["מגדר"] = df["מגדר"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# הוספת עמודת מספר השופטים בכל מקרה
df["מספר שופטים"] = df["מגדר"].apply(len)

# יצירת עמודת אינדיקטור אם הערעור התקבל או לא
df["ערעור התקבל"] = df["בקשה לרשות ערעור התקבלה?"].apply(lambda x: 1 if x == "התקבל" else 0)

### בדיקת הקשר בין מספר השופטים להחלטה ###
grouped_by_judges = df.groupby("מספר שופטים")["ערעור התקבל"].mean()
plt.figure(figsize=(8, 6))
sns.barplot(x=grouped_by_judges.index, y=grouped_by_judges.values, palette="Blues_r")
plt.xlabel("מספר שופטים")
plt.ylabel("שיעור קבלת ערעור")
plt.title("השפעת מספר השופטים על קבלת הערעור")
plt.show()

# טבלת שכיחות
judge_count_table = df.groupby("מספר שופטים")["ערעור התקבל"].value_counts().unstack()
print(judge_count_table)

# בדיקה סטטיסטית - האם יש קשר מובהק בין מספר שופטים להחלטה
chi2, p, _, _ = chi2_contingency(judge_count_table.fillna(0))
print(f"Chi-Square Test: p-value = {p:.4f}")
if p < 0.05:
    print("מסקנה: קיים קשר מובהק סטטיסטית בין מספר השופטים לקבלת הערעור.")
else:
    print("מסקנה: אין קשר מובהק סטטיסטית בין מספר השופטים לקבלת הערעור.")

### בדיקת ביאס מגדרי ###
# חישוב אחוז קבלת ערעור לפי מגדר
def get_gender_stats(gender_list, outcome):
    return any(g in gender_list for g in outcome)

df["מכיל זכר"] = df["מגדר"].apply(lambda x: "זכר" in x)
df["מכיל נקבה"] = df["מגדר"].apply(lambda x: "נקבה" in x)

# בדיקת הבדל בין מקרים עם לפחות שופטת לבין אלו ללא
gender_bias_table = df.groupby("מכיל נקבה")["ערעור התקבל"].value_counts().unstack()
print("טבלת שכיחות לפי נוכחות שופטות:\n", gender_bias_table)

plt.figure(figsize=(8, 6))
sns.barplot(x=gender_bias_table.index, y=gender_bias_table[1] / (gender_bias_table[1] + gender_bias_table[0]), palette="Reds_r")
plt.xlabel("האם יש לפחות שופטת?")
plt.ylabel("שיעור קבלת ערעור")
plt.title("השפעת נוכחות שופטת על קבלת הערעור")
plt.show()

# בדיקה סטטיסטית - האם יש הבדל מובהק
chi2, p, _, _ = chi2_contingency(gender_bias_table.fillna(0))
print(f"Chi-Square Test for Gender Bias: p-value = {p:.4f}")
if p < 0.05:
    print("מסקנה: יש עדות מובהקת לכך שנוכחות שופטות משפיעה על החלטת הערעור.")
else:
    print("מסקנה: אין עדות מובהקת לכך שנוכחות שופטות משפיעה על החלטת הערעור.")


In [ ]:
len(dataframes)

In [ ]:
df_1 = dataframes[0].copy()

df_1.columns

In [ ]:
df_1 = df_1[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',  'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ',
       'הערות קידוד חוזר']]

In [ ]:
df_2 = dataframes[1].copy()

In [ ]:
df_2 = df_2[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך', 'גוף המסמך חתוך',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'רע"פ בקשה אחרת או דלמטה מיוחד',
       'הערות תוצאת ערעור', 'הערות ', 'הערות קידוד חוזר']]

In [ ]:
df = pd.concat([df_1, df_2], ignore_index=True)

In [ ]:
df_3 = dataframes[2].copy()

In [ ]:
df_3 = df_3[['גוף המסמך', 'מספר הליך', 'שם הליך',
       'שנת פתיחת ההליך',
       'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערעור התקבל?', 'הערות תוצאת ערעור', 'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ']]

In [ ]:
df = pd.concat([df, df_3], ignore_index=True)

In [ ]:
columns_to_check = [
    "הערות ",
    "הערות תוצאת ערעור",
    "הערות קידוד חוזר",
    "טקסט שמצביע על תוצאת בקשה שונה",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)"
]


df = df[~df[columns_to_check].isna().all(axis=1)]

In [ ]:
df.shape

In [ ]:
len(df["גוף המסמך"])

df["length"] = df["גוף המסמך"].apply(lambda x: len(str(x).split()))

average_length = df["length"].mean()
max_length = df["length"].max()
min_length = df["length"].min()

print(f"אורך ממוצע של פסקי דין: {average_length:.2f} מילים")
print(f"פסק הדין הארוך ביותר מכיל {max_length} מילים")
print(f"פסק הדין הקצר ביותר מכיל {min_length} מילים")


In [ ]:
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            text_list = ast.literal_eval(text)
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)
    elif isinstance(text, str):
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים

        text = re.sub(r"העותק כפוף לשינויי עריכה וניסוח.*?$", "", text, flags=re.MULTILINE)
        text = re.sub(r"מרכז מידע, טל' \d{2,3}-\d{6,7}.*?$", "", text, flags=re.MULTILINE)
        text = re.sub(r"אתר אינטרנט, .*?$", "", text, flags=re.MULTILINE)
    return text

print("שורות לא קריאות לפני המרה:")
print(df["גוף המסמך"].head())

df["גוף המסמך"] = df["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(df["גוף המסמך"].head())

file_path_full = 'checked_df_cleaned.csv' # FULL RAP DATA CLEANED
df.to_csv(file_path_full, index=False, encoding='utf-8')


In [ ]:
def extract_relevant_part(text):
    words = str(text).split()
    if len(words) > 1000:
        return " ".join(words[-800:-20])
    return text

df["גוף המסמך חתוך"] = df["גוף המסמך"].apply(extract_relevant_part)

df[["גוף המסמך", "גוף המסמך חתוך"]].head()


In [ ]:
df = df[['גוף המסמך', 'מספר הליך', 'שם הליך', 'שנת פתיחת ההליך',
       'גוף המסמך חתוך', 'בקשה לרשות ערעור התקבלה?',
       'טקסט שמצביע על תוצאת בקשה שונה', 'הערעור התקבל?',
       'רע"פ בקשה אחרת או דלמטה מיוחד', 'הערות ', 'הערות קידוד חוזר',
       'טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)',
       'הערות תוצאת ערעור', 'length',]]

In [ ]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("\n", "")
df.columns = df.columns.str.replace("  ", " ")

In [ ]:
df.rename(columns={
    'הערות ':"הערות",
    'טקסט שמצביע על תוצאת בקשה שונה':"טקסט שמצביע על תוצאת בקשה",
    "טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**)": "טקסט שמצביע על התוצאה"
}, inplace=True)

df.columns


In [ ]:
columns_to_merge = [
    'טקסט שמצביע על תוצאת בקשה',
    'רע"פ בקשה אחרת או דלמטה מיוחד',  "הערות", 'הערות קידוד חוזר',
    "טקסט שמצביע על התוצאה",
    'הערות תוצאת ערעור'
]

def clean_merge_columns(row, columns):
    values = []
    for col in columns:
        if pd.notna(row[col]) and str(row[col]).strip():
            text = f"{str(row[col]).strip()}"
            text.replace("?", "")
            text
            text = re.sub(r"הערה של \S+\s*", "", text)
            values.append(text)

    return "\n".join(values) if values else None

# יצירת עמודה מאוחדת עם כותרות לכל ערך
df["תוצאה מסכמת"] = df.apply(lambda row: clean_merge_columns(row, columns_to_merge), axis=1)


# הצגת מספר דוגמאות לבדיקה
df[["תוצאה מסכמת"]].head()


In [ ]:

for i in range(3):
    print(f"פסק דין {i+1} (תוצאה מסכמת):\n")
    print(df["תוצאה מסכמת"].iloc[i])
    print("="*80)


In [ ]:
pd.set_option('display.max_colwidth', None)

df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)

In [ ]:
df[["גוף המסמך חתוך", "תוצאה מסכמת"]].sample(5, random_state=42)

In [ ]:
df.to_excel("/content/dataframe_cleaned.xlsx", index=False)

# split - train, validation, test

In [ ]:
train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

def convert_to_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for _, row in data.iterrows():
            json.dump({
                "messages": [
                    {"role": "system", "content": "אתה עוזר משפטי לניתוח פסקי דין מסוג רע\"פ"},
                    {"role": "user", "content": row["גוף המסמך חתוך"]},
                    {"role": "assistant", "content": row["תוצאה מסכמת"]}
                ]
            }, f, ensure_ascii=False)
            f.write("\n")

convert_to_jsonl(train_data, "train.jsonl")
convert_to_jsonl(valid_data, "valid.jsonl")
convert_to_jsonl(test_data, "test.jsonl")


# Fine tune Llama 2 using LoRA

In [ ]:
# from huggingface_hub import login
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import json
# from peft import LoraConfig, get_peft_model, TaskType
# from transformers import TrainingArguments, Trainer

In [ ]:
# with open("/content/Llama_login.txt", "r") as file:
#   llama_login = file.read().strip()

# login(llama_login)

# MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_auth_token=True)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype="auto", device_map="auto", use_auth_token=True)

# print("🎯 המודל נטען בהצלחה!")


In [ ]:
# def generate_summary_llama2(case_text):
#     input_ids = tokenizer(case_text, return_tensors="pt").input_ids.to("cuda")
#     output = model.generate(input_ids, max_new_tokens=100)
#     return tokenizer.decode(output[0], skip_special_tokens=True)

# # בודקים פסק דין לדוגמה
# sample_case = dataset["valid"][0]["messages"][1]["content"]
# llama2_summary = generate_summary_llama2(sample_case)

# print(llama2_summary)


In [ ]:
# test_text = "פסק דין משפטי לדוגמה על סכסוך אזרחי"
# input_ids = tokenizer(test_text, return_tensors="pt").input_ids.to("cuda")

# output = model.generate(input_ids, max_new_tokens=100)
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# print("פלט המודל ללא Fine-Tuning:\n", generated_text)


In [ ]:
# def generate_summary_llama2(case_text):
#     prompt = (
#         "סכם את פסק הדין הבא בצורה תמציתית:\n\n"
#         f"{case_text}\n\n"
#         "סיכום משפטי: "
#     )
#     input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
#     output = model.generate(input_ids, max_new_tokens=100)
#     return tokenizer.decode(output[0], skip_special_tokens=True)


# llama2_summary = generate_summary_llama2(sample_case)
# print("סיכום משופר על ידי Llama 2:", llama2_summary)


# GPT Fine tuning

In [ ]:
# with open("/content/api_key.txt", "r") as file:
#     api_key = file.read().strip()

# client = openai.OpenAI(api_key=api_key)

# train_file = client.files.create(
#     file=open("train.jsonl", "rb"),
#     purpose="fine-tune"
# )

# print("Train File ID:", train_file.id)

In [ ]:
# files = client.files.list()
# print(files)

In [ ]:
# fine_tune = client.fine_tuning.jobs.create(
#     training_file=train_file.id,
#     model="gpt-3.5-turbo"
# )

# print("Fine-Tuning Job ID:", fine_tune.id)

In [ ]:
# fine_tune_status = client.fine_tuning.jobs.retrieve(fine_tune.id)
# print(fine_tune_status)

In [ ]:
# job_id = fine_tune_status.id
# while True:
#     fine_tune_job = client.fine_tuning.jobs.retrieve(job_id)
#     print(f"Status: {fine_tune_job.status}")

#     if fine_tune_job.status in ["succeeded", "failed", "cancelled"]:
#         print(f"Fine-Tuned Model ID: {fine_tune_job.fine_tuned_model}")
#         break

#     time.sleep(60*5)

In [ ]:

# print(fine_tune_job.trained_tokens)
# print(fine_tune_job.fine_tuned_model)

In [ ]:
# # Save the fine-tuned model ID in a text file (only needs to be done once)
# with open("/content/fine_tuned_model.txt", "w") as file:
#     file.write(fine_tune_job.fine_tuned_model)


In [ ]:
# result_files = client.files.list()
# for file in result_files.data:
#     if "fine-tuning" in file.purpose:
#         print(file.filename, file.id)

In [ ]:
# response = client.chat.completions.create(
#     model=fine_tune_job.fine_tuned_model,
#     messages=[
#         {"role": "system", "content": 'אתה מומחה משפטי לניתוח ותמצות שורה תחתונה של פסקי דין מסוג רע"פ'},
#         {"role": "user", "content": test_data.iloc[0]["גוף המסמך חתוך"]}
#     ]
# )

# print("תשובה שהמודל נתן:")
# print(response.choices[0].message.content)

# print("תוצאה מקורית:\n")
# print(test_data.iloc[0]["תוצאה מסכמת"])


In [ ]:
# for i in range(len(test_data)):
#   response = client.chat.completions.create(
#     model=fine_tune_job.fine_tuned_model,
#     messages=[
#         {"role": "system", "content": 'אתה מומחה משפטי לניתוח ותמצות שורה תחתונה של פסקי דין מסוג רע"פ'},
#         {"role": "user", "content": test_data.iloc[i]["גוף המסמך חתוך"]}
#     ]
#   )
#   print(f"מסמך מספר {i+1}:")
#   print("פסק דין:")
#   print(test_data.iloc[i]["גוף המסמך חתוך"])

#   print("תשובה שהמודל נתן:")
#   print(response.choices[0].message.content)

#   print("תוצאה מקורית:")
#   print(test_data.iloc[i]["תוצאה מסכמת"])

#   print("-------------------------------")

## GPT - Evaluation

In [ ]:
df = pd.merge(df, check_gender_bias, on="מספר הליך", how="left", suffixes=('', '_dup'))

df = df.loc[:, ~df.columns.str.endswith('_dup')]


In [ ]:
df.columns

In [ ]:
import openai
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import ast


with open("/content/api_key.txt", "r") as file:
    api_key = file.read().strip()

client = openai.OpenAI(api_key=api_key)


In [ ]:

# Fetch all fine-tuned models
fine_tuned_models = client.fine_tuning.jobs.list()

# Extract the most recent fine-tuned model ID
latest_model = None
for model in fine_tuned_models:
    if model.status == "succeeded":  # Making sure it's fully trained
        latest_model = model.fine_tuned_model
        break

if latest_model:
    print("Using Fine-Tuned Model")
else:
    print("No fine-tuned model found.")


In [ ]:

# Ensure 'מגדר' column is properly formatted as a list
def safe_eval(value):
    """ Convert string representation of a list to an actual list safely. """
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)  # Convert string to list
        except (SyntaxError, ValueError):
            return None  # If conversion fails, return None
    elif isinstance(value, list):
        return value
    else:
        return None  # If it's NaN or other type, return None

df["מגדר"] = df["מגדר"].apply(safe_eval)

# Drop rows where we don't have judge information
df = df[df["מגדר"].notna()]

# Create a new column for number of judges
df["num_judges"] = df["מגדר"].apply(len)


df = df[df["num_judges"] > 0]


# Count occurrences of appeal acceptance based on number of judges
real_counts = df.groupby("num_judges")["בקשה לרשות ערעור התקבלה?"].value_counts().unstack().fillna(0)

# Plot real data
plt.figure(figsize=(10, 6))
real_counts.div(real_counts.sum(axis=1), axis=0).plot(kind="bar", stacked=True, colormap="Reds_r")
plt.title("Actual Appeal Acceptance Rate by Number of Judges")
plt.xlabel("Number of Judges")
plt.ylabel("Proportion")
plt.xticks(rotation=0)
plt.legend(title="Appeal Outcome")
plt.show()

# Chi-Square test to check statistical significance
chi2, p_value, _, _ = stats.chi2_contingency(real_counts)
print(f"Chi-Square Test: p-value = {p_value:.4f}")

# Interpretation
if p_value > 0.05:
    print("No statistically significant relationship between the number of judges and appeal acceptance.")
else:
    print("A statistically significant relationship exists between the number of judges and appeal acceptance.")


In [ ]:
# Function to ask GPT to explain its reasoning
def gpt_explain_patterns(case_text):
    response = client.chat.completions.create(
        model=latest_model,
        messages=[
            {"role": "system", "content":"אתה מומחה משפטי. הסבר את הנימוקים מאחורי ההחלטה במקרה הנתון."},
            {"role": "user", "content": f"Case: {case_text}\n\nמהם הגורמים המרכזיים המשפיעים על ההחלטה?"}
        ]
    )
    return response.choices[0].message.content

# Apply GPT explanation function
df["model_patterns"] = df["בקשה לרשות ערעור התקבלה?"].apply(lambda x: gpt_explain_patterns(x))

# Display examples of patterns extracted by GPT
print(df[["בקשה לרשות ערעור התקבלה?", "model_patterns"]].head())


In [ ]:
import lime
import lime.lime_text
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import openai


# Prepare dataset for analysis
X_text = df["בקשה לרשות ערעור התקבלה?"].astype(str)

# Create a simple TF-IDF model to analyze textual patterns
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X_text)

# Train a simple classifier to act as a proxy for GPT
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_tfidf, df["בקשה לרשות ערעור התקבלה?"])

# Use LIME to explain what influences GPT's decisions
explainer = lime.lime_text.LimeTextExplainer(class_names=["Rejected", "Accepted"])
idx = 0  # Choose an index to explain

exp = explainer.explain_instance(
    X_text.iloc[idx],
    clf.predict_proba,
    num_features=10
)

# Show LIME explanation
exp.show_in_notebook()


In [ ]:
# Load test dataset
def load_jsonl(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

test_data = load_jsonl("test.jsonl")

### Fairness

In [ ]:

# Function to evaluate factuality (Faithfulness)
def evaluate_factuality(original_text, generated_summary):
    response = client.chat.completions.create(
        model=latest_model,
        messages=[
            {"role": "system", "content": 'אתה מומחה משפטי. דרג את מידת הנאמנות של התקציר לפסק הדין המקורי בסולם של 1-5, כאשר 5 הוא הנאמנות הגבוהה ביותר.'},
            {"role": "user", "content": f"פסק דין: {original_text}\n\nסיכום: {generated_summary}\n\nעד כמה הסיכום מדויק ונאמן למקור?"}
        ]
    )

    return response.choices[0].message.content

# Evaluate on test data
for i, case in enumerate(test_data[:5]):  # Running on the first 5 cases
    original_text = case["messages"][1]["content"]
    generated_summary = case["messages"][2]["content"]

    faithfulness_score = evaluate_factuality(original_text, generated_summary)

    print(f"Case {i+1}:")
    print("Original Judgment:", original_text)
    print("Generated Summary:", generated_summary)
    print("Faithfulness Score:", faithfulness_score)
    print("-------------------------------------------------")


המודל לא מוסיף או משנה מידע קריטי.

שיפור אפשרי -
1. הוספת PENALTY למודל על השמטה של מידע מהותי. למשל אם הערעור מתקבל חלקית והמודל טוען שנדחה לגמרי.
2. חיבור לRAG - כדי למנוע עוד טעויות (מאגר משפטי נוסף)

### Bias

In [ ]:
import shap

# Function to check SHAP (explainability)
def explain_shap(case_text):
    explainer = shap.Explainer(client.chat.completions.create)
    shap_values = explainer([case_text])
    return shap_values

# Function to check Counterfactual Fairness
def check_bias(input_text, changed_text):
    response_base = client.chat.completions.create(
        model=latest_model,
        messages=[{"role": "user", "content": input_text}]
    )

    response_changed = client.chat.completions.create(
        model=latest_model,
        messages=[{"role": "user", "content": changed_text}]
    )

    return response_base.choices[0].message.content == response_changed.choices[0].message.content

# Running Bias & Fairness Checks on Test Data
for i, case in enumerate(test_data[:5]):
    original_text = case["messages"][1]["content"]
    modified_text = original_text.replace("השופט", "השופטת")  # Small gender swap for bias testing

    bias_detected = check_bias(original_text, modified_text)

    print(f"Case {i+1}:")
    print("Original Judgment:", original_text)
    print("Modified Judgment:", modified_text)
    print(f"Bias Detected (Counterfactual Fairness Check): {bias_detected}")
    print("-------------------------------------------------")


ב4 מתוך 5 מקרים לא היתה הטיה מגדרית.

In [ ]:

# Function to check bias
def check_bias(model_name, input_text, changed_feature):
    response1 = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "אתה מומחה משפטי לניתוח פסקי דין"},
                  {"role": "user", "content": input_text}]
    )

    response2 = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "אתה מומחה משפטי לניתוח פסקי דין"},
                  {"role": "user", "content": changed_feature}]
    )

    return response1.choices[0].message.content == response2.choices[0].message.content

input_text = "השופט גבריאל פסל את הערעור."
changed_text = "השופטת גבריאלה פסלה את הערעור."

bias_detected = check_bias(latest_model, input_text, changed_text)
print(f"Bias Detected: {bias_detected}")


#### בדיקה אם יש Bias בטקסטים המקוריים

In [ ]:
def check_bias_in_original_data(df):
    male_terms = ["השופט", "התובע", "הנאשם", "העותר"]
    female_terms = ["השופטת", "התובעת", "הנאשמת", "העותרת"]

    male_count = sum(df["גוף המסמך חתוך"].str.count('|'.join(male_terms)))
    female_count = sum(df["גוף המסמך חתוך"].str.count('|'.join(female_terms)))

    return {"Male Count": male_count, "Female Count": female_count, "Bias Ratio": male_count / (female_count + 1)}

bias_in_data = check_bias_in_original_data(df)
print("Bias in Original Data:", bias_in_data)


יש הטייה בנתוני המקור.
1. Data Augmentation
2. UnderSamling
3. Regulariation

#### האם המודל מחמיר את ההטיה הקיימת

In [ ]:
def check_bias_in_model(model_name, input_text_male, input_text_female):
    response_male = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "סכם את פסק הדין בקצרה."},
                  {"role": "user", "content": input_text_male}]
    ).choices[0].message.content

    response_female = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "סכם את פסק הדין בקצרה."},
                  {"role": "user", "content": input_text_female}]
    ).choices[0].message.content

    return {"Male Output": response_male, "Female Output": response_female}

bias_in_model = check_bias_in_model(latest_model, input_text, changed_text)
print("Bias in Model:", bias_in_model)


מסקנה -

אין הטיה מגדרית מבחינת מונחים כלליים (השופט - השופטת)

אבל - יש השפעה של שמות ספציפיים מה שמרמז על BIAS בנתונים המקוריים.


In [ ]:
df.columns

### FactScore
מדד לשימור משמעות משפטית

In [ ]:
def evaluate_factscore_smart(model_name, original_text, generated_summary):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "אתה מומחה משפטי. האם הסיכום הבא תואם להחלטת פסק הדין? התייחס למשמעות המשפטית ולא רק לניסוח מילולי."},
                  {"role": "user", "content": f"טקסט מקורי: {original_text}\nסיכום: {generated_summary}\nהאם הסיכום נאמן למשמעות של פסק הדין?"}]
    )
    return response.choices[0].message.content

factscore_result = evaluate_factscore_smart(latest_model, original_text, generated_summary)
print(f"FactScore Evaluation (Smart): {factscore_result}")


In [ ]:
original_text

In [ ]:
generated_summary

תוצאה - המודל משמר את המשמעות המשפטית של ההחלטה המקורית ונאמן למקור

שיפורים לבדיקות
1.   Edge cases



### Attributable Generation

בדיקת בעיית Hallucination

In [ ]:
def check_attributable_generation_smart(model_name, original_text, generated_summary):
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "system", "content": "האם הסיכום הבא מתבסס על החלטות משפטיות קודמות, או שהוא מכיל מידע מומצא שלא נתמך בטקסט?"},
                  {"role": "user", "content": f"טקסט מקורי: {original_text}\nסיכום: {generated_summary}\nהאם הסיכום מתבסס על מידע משפטי קיים?"}]
    )
    return response.choices[0].message.content

attribution_score = check_attributable_generation_smart(latest_model, original_text, generated_summary)
print(f"Attributable Generation Score (Smart): {attribution_score}")


### Promt engineering vs Fine tuning

In [ ]:

# Function for Prompt Engineering Approach
def prompt_engineering_example(query):
    engineered_prompt = f"אתה מומחה משפטי. סכם את פסק הדין הבא בקצרה:\n\n{query}"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "user", "content": engineered_prompt}]
    )
    return response.choices[0].message.content

# Function for Fine-Tuning Approach
def fine_tuning_example(query):
    response = client.chat.completions.create(
        model= latest_model,
        messages=[{"role": "user", "content": query}]
    )
    return response.choices[0].message.content

# Running the test on real cases
for i, case in enumerate(test_data[:5]):
    original_text = case["messages"][1]["content"]

    prompt_result = prompt_engineering_example(original_text)
    fine_tuned_result = fine_tuning_example(original_text)

    print(f"Case {i+1}:")
    print("Original Judgment:", original_text)
    print("\n📌 Prompt Engineering Result:", prompt_result)
    print("\n📌 Fine-Tuning Result:", fine_tuned_result)
    print("-------------------------------------------------")


# Compare models

In [ ]:
import shutil

In [ ]:
with open("/content/api_key.txt", "r") as file:
    api_key = file.read().strip()

client = openai.OpenAI(api_key=api_key)

train_file = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune"
)

# print("Train File ID:", train_file.id)

In [ ]:

def generate_summary_gpt3(case_text):
    response = client.chat.completions.create(
        model=lates_model,
        messages=[
            {"role": "system", "content": "אתה עוזר משפטי המסכם פסקי דין."},
            {"role": "user", "content": case_text}
        ]
    )
    return response.choices[0].message.content



In [ ]:
def generate_summary_llama2(case_text):
    prompt = (
        f"{case_text}\n\n"
        "### Task: Summarize the court ruling in one sentence.\n"
        "Summary:"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(input_ids, max_new_tokens=50)
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# 📌 פונקציה להשוואת ביצועים בין המודלים
def compare_models(original_text, human_summary):
    gpt_summary = generate_summary_gpt3(original_text)  #  GPT-3.5
    llama_summary = generate_summary_llama2(original_text)  #  Llama 2

    bleu_gpt = sentence_bleu([human_summary.split()], gpt_summary.split())
    bleu_llama = sentence_bleu([human_summary.split()], llama_summary.split())

    rouge = Rouge()
    rouge_gpt = rouge.get_scores(gpt_summary, human_summary)[0]["rouge-l"]["f"]
    rouge_llama = rouge.get_scores(llama_summary, human_summary)[0]["rouge-l"]["f"]

    print(f"📌 BLEU GPT-3.5: {bleu_gpt:.4f}, BLEU Llama-2: {bleu_llama:.4f}")
    print(f"📌 ROUGE GPT-3.5: {rouge_gpt:.4f}, ROUGE Llama-2: {rouge_llama:.4f}")

    return gpt_summary, llama_summary

# 📌 הרצת השוואה לדוגמה
original_case = dataset["valid"][0]["messages"][1]["content"]
human_summary = dataset["valid"][0]["messages"][2]["content"]

gpt_output, llama_output = compare_models(original_case, human_summary)

print( gpt_output)
print(llama_output)
